In [2]:
:dep bytes = {version = "0.5.2"}
:dep rand = {version = "0.7.2"}


In [3]:
extern crate bytes;
extern crate rand;
use bytes::{BytesMut, BufMut};
use rand::Rng;
use rand::rngs::ThreadRng;

In [5]:


pub struct SwitchStatus{
    random: RandSwitchMetric,
    volts: u8,
    temp: u8,
    detd: u8,
    perror: u8,
    cerror: u8,
    nfire: u8,
    flags: u8,
}

impl SwitchStatus{
    pub fn new() -> Self{
        let mut rng = RandSwitchMetric::new();
        SwitchStatus{
            random: rng,
            volts: 0,
            temp: 0,
            detd: 0,
            perror: 0,
            cerror: 0,
            nfire: 0,
            flags: 0,
        }
    }
    
    pub fn generate(&mut self) -> BytesMut{
        let mut packet = BytesMut::with_capacity(7);
        
        let volt = self.random.voltage();
        let temp = self.random.temp();
        
        packet.put_u8(volt);
        packet.put_u8(temp);
        packet.put_u8(self.detd);
        packet.put_u8(self.perror);
        packet.put_u8(self.cerror);
        packet.put_u8(self.nfire);
        packet.put_u8(self.random.rand());
        
        packet
    }
    
    
    pub fn chksum_error(&mut self, amt: u8){
        self.cerror += amt;
    }
    
    pub fn packet_errors(&mut self, amt: u8){
        self.perror += amt;
    }
    
}


#[derive(Debug)]
pub struct RandSwitchMetric {
    rng: ThreadRng,
}

impl RandSwitchMetric {
    pub fn new() -> Self {
        RandSwitchMetric {
            rng: rand::thread_rng(),
        }
    }

    pub fn voltage(&mut self) -> u8 {
        self.rng.gen_range(50 as u8, 60 as u8)
    }
    
    pub fn temp(&mut self) -> u8{
        self.rng.gen_range(20 as u8, 28 as u8)
    }
    
    pub fn between(&mut self, low: u8, high: u8)->u8{
        self.rng.gen_range(low, high)
    }
    
    pub fn rand(&mut self) -> u8{
        self.rng.gen()
    }
    
    
}

let mut switch_status = SwitchStatus::new();
let packet = switch_status.generate();

println!("{:?}", packet);

// let mut r = RandSwitchMetric::new();
// for _  in 0..10{
//     println!("{}V", r.random_voltage());
// }

b":\x1b\0\0\0\0\xc6"


In [18]:

fn split_packet(mut packet: &mut BytesMut) -> (BytesMut, BytesMut, BytesMut){
    let addr = packet.split_to(2);
    let chksum = packet.split_off(packet.len()-1);
    let body = packet.clone();
    (addr, body, chksum)
}


fn check_chksum(packet: &BytesMut) -> bool {
    let mut chksum: u8 = 0;

    for (idx, byte) in packet.iter().enumerate() {
        if idx == packet.len() - 1 {
            break;
        }

        chksum ^= byte;
    }
    chksum == packet[packet.len() - 1]
}


// let result = split_packet(&mut packet);
// let mut chksum: u8 = 0;
// for (idx, byte) in packet.iter().enumerate(){
    
//     if idx == packet.len()-1{
//         break;
//     }
//     chksum ^= byte;
// //     println!("{:?}", byte);
// }

// println!("{}",chksum == packet[packet.len()])
// println!("{:x?}, {:x?}, compare: {}", chksum, packet[packet.len()-1], chksum==packet[packet.len()-1]);

In [39]:
#[derive(Debug)]
pub enum Commands {
    ACK,
    NACK,
    GetStatus,
    GoIdle,
    InvalidCmd,
}

impl Commands {
    pub fn value(&self) -> BytesMut {
        match *self {
            Commands::ACK => BytesMut::from(&b"\x06"[..]),
            Commands::NACK => BytesMut::from(&b"\x15"[..]),
            Commands::GetStatus => BytesMut::from(&b"\x05"[..]),
            Commands::GoIdle => BytesMut::from(&b"\x1e"[..]),
            Commands::InvalidCmd => BytesMut::from(&b"\x00"[..]),
        }
    }
    
    fn from_bytes(bytes: &[u8]) -> Self {
        match bytes {
            b"\x05" => Commands::ACK,
            _ => panic!("Can't find a match!"),
        }
    }
    

}


In [40]:
fn gen_chksum(packet: &BytesMut)-> BytesMut{
    let mut chksum = BytesMut::from(&b"\x00"[..]);

    for (idx, byte) in packet.iter().enumerate() {
        if idx == packet.len() - 1 {
            break;
        }

        chksum[0] ^= byte;
    }
    
    BytesMut::from(&chksum[..])
}

fn gen_command(cmd: &BytesMut)-> BytesMut{
    let mut packet = BytesMut::new();
    
    let addr: u16 = 0xff1a;
    let addr: BytesMut = BytesMut::from(&addr.to_be_bytes()[..]);
    
    packet.extend_from_slice(&addr[..]);
    packet.extend_from_slice(&cmd[..]);
    
    
    let chksum = gen_chksum(&packet);
    packet.extend_from_slice(&chksum[..]);
    packet
}


In [58]:
let mut packet = gen_command(&Commands::GetStatus.value());

fn parse_packet(mut input: &mut BytesMut) -> Commands{
    
    let (addr, body, chksum) = split_packet(&mut input);
    
    let result = Commands::from_bytes(&body);
    
    result
}

let result = parse_packet(&mut packet);
(result.value(), result)

(b"\x06", ACK)

In [68]:
fn match_str(s: &'static str){
    match *s{
        "yay" => println!("yay"),
        "no" => println!("no"),
        _ => println!("Other"),
    };
    
}

Error: mismatched types

Error: mismatched types